In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path
from unidecode import unidecode

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from mlens.ensemble import SuperLearner


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score


[MLENS] backend: threading


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63949%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=B0FtI9sjXGqVa4sTOkH2gbRak6mhWs&access_type=offline
Descargando sistemas10.csv: 100%
Descargando sistemas6.csv: 100%
Descargando sistemas2.csv: 100%
Descargando sistemas7.csv: 100%
Descargando sistemas4.csv: 100%
Descargando sistemas9.csv: 100%
Descargando sistemas5.csv: 100%
Descargando sistemas3.csv: 100%
Descargando sistemas8.csv: 100%
Descargando electrica10.csv: 100%
Descargando electrica9.csv: 100%
Descargando electrica8.csv: 100%
Descargando electrica7.csv: 100%
Descargando electrica6.csv: 100%
Descargando electrica5.csv: 100%
Descargando electrica4.csv: 100%
Descargando electrica3.csv: 100%
Descargando electrica2.csv: 100%
Descargando catastral2.csv: 100%
Descargando cata

In [3]:
"""
variables_por_carrera = {
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO', 'PROMEDIO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS', 'PROMEDIO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO', 'PROMEDIO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO', 'PROMEDIO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS', 'PROMEDIO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE', 'PROMEDIO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO', 'PROMEDIO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE', 'PROMEDIO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ', 'PROMEDIO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}
"""

"\nvariables_por_carrera = {\n    'industrial': {\n        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO', 'PROMEDIO_UNO'],\n        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS', 'PROMEDIO_DOS'],\n        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES', 'PROMEDIO_TRES'],\n        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NO

In [3]:
variables_por_carrera = { 
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'sistemas': {
        '1': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'QUIMICA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'NOTA_DIFERENCIAL', 'NOTA_PROG_BASICA', 'NOTA_CATEDRA_FJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_CATEDRA_DEM', 'NOTA_CATEDRA_CON', 'NOTA_LOGICA', 'RENDIMIENTO_DOS'],
        '3': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'RENDIMIENTO_TRES'],
        '4': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'IDIOMA_ICFES', 'PROMEDIO_UNO', 'PROMEDIO_DOS', 'PROMEDIO_TRES', 'NOTA_PROG_BASICA', 'NOTA_TEXTOS', 'NOTA_CATEDRA_DEM', 'NOTA_INTEGRAL', 'NOTA_PROG_ORIENTADA', 'NOTA_ETICA', 'NOTA_FISICA_DOS', 'NOTA_TGS', 'NOTA_PROG_AVANZADA', 'RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ECONOMIA_UNO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISE O', 'NOTA_EI_TRES', 'RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES', 'NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'catastral': {
        '1': ['variable1_catastral', 'variable2_catastral', 'variable3_catastral'],
        '2': ['variable4_catastral', 'variable5_catastral', 'variable6_catastral']
    }
}

In [4]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MÓDULO ANALÍTICA PREDICTIVA/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="1"
modelo="CatBoostClassifier"
modelos_seleccionados = ['KNeighbors', 'RedNeuronal']  # Lista de modelos que quieres incluir


In [5]:
datos = cargar_datos(carrera, semestre)
columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD,RENDIMIENTO_UNO
0,474,65,73,75,79,20,2
1,523,99,69,78,60,1,2
2,483,63,76,82,77,20,1
3,528,88,69,78,75,19,1
4,478,78,69,66,69,8,2
...,...,...,...,...,...,...,...
2003,342,68,0,0,58,11,2
2004,347,69,0,0,71,19,2
2005,348,67,0,0,68,11,1
2006,340,70,0,0,63,11,2


In [6]:
def numero_a_letras(numero):
    numeros_letras = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']
    return numeros_letras[int(numero)]

semestre_en_letras = numero_a_letras(semestre)
print(semestre_en_letras)


uno


In [7]:
X = df.loc[:, ~df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]
Y = df.loc[:, df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]                                                     
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2008, 6) (2008, 1)


In [8]:
#X = X.astype('float32')                         
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2008, 6) (2008,)


In [9]:
from collections import Counter
frecuencias = Counter(Y)
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")

El valor 1 aparece 1249 veces
El valor 0 aparece 675 veces
El valor 2 aparece 83 veces
El valor 3 aparece 1 veces


### TRANSFORMACIÓN DE DATOS

In [10]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.598 -0.897  1.005  1.046  1.268  1.353]
 [ 2.502  2.983  0.97   1.072 -0.304 -2.363]
 [ 1.767 -1.135  1.031  1.104  1.099  1.353]
 [ 2.592  1.758  0.97   1.072  0.931  1.202]
 [ 1.673  0.621  0.97   0.965  0.432 -0.682]]


,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,QUIMICA_ICFES,IDIOMA_ICFES,LOCALIDAD
0,1.597977,-0.896979,1.005474,1.046461,1.268136,1.353466
1,2.501943,2.982940,0.970095,1.071635,-0.304403,-2.362712


In [12]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Crear una instancia de RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias
Xpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)

# Imprimir la distribución de clases después del oversampling
print("Distribución de clases después del oversampling:", Counter(Y))
"""


'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\n# Crear una instancia de RandomOverSampler\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\n# Aplicar RandomOverSampler para generar muestras sintéticas de las clases minoritarias\nXpandas_T_JOHNSON1, Y = ros.fit_resample(Xpandas_T_JOHNSON1, Y)\n\n# Imprimir la distribución de clases después del oversampling\nprint("Distribución de clases después del oversampling:", Counter(Y))\n'

In [12]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1405 datos para entrenamiento y 603 datos para prueba


In [14]:
"""
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

ros = RandomOverSampler(sampling_strategy='auto', random_state=42)

X_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)

print("Distribución de clases después del oversampling:", Counter(Y_trn))

"""

'\nfrom imblearn.over_sampling import RandomOverSampler\nfrom sklearn.model_selection import train_test_split\n\nros = RandomOverSampler(sampling_strategy=\'auto\', random_state=42)\n\nX_trn,Y_trn = ros.fit_resample(X_trn, Y_trn)\n\nprint("Distribución de clases después del oversampling:", Counter(Y_trn))\n\n'

In [15]:
from collections import Counter
frecuencias = Counter(Y_trn)   
for valor, frecuencia in frecuencias.items():
    print(f"El valor {valor} aparece {frecuencia} veces")   

El valor 1 aparece 861 veces
El valor 0 aparece 477 veces
El valor 2 aparece 66 veces
El valor 3 aparece 1 veces


### KNEIGHBORSCLASSIFIER

In [16]:
# conocer los hiperparámetro de cada algoritmo
estimator = KNeighborsClassifier()
estimator.get_params()


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [13]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(1, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto', 'kd_tree','ball_tree','brute'],
        #'p': [i for i in range(1, 18,1)],
        'weights': ['uniform']
        
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_knn=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_
X_trn = X_trn
Y_trn = Y_trn 
modelo_knn, mejores_hiperparametros_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_knn

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.427558257345495
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'}


{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 15,
 'weights': 'uniform'}

In [14]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_knn.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_entrenamiento = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_entrenamiento["MODELO"]='KNeighbors'
resultados_df_knn_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_knn_entrenamiento


Precisión:  60.19
Exhaustividad:  64.56
Puntuación F1:  59.44
Exactitud:  64.56
Información Mutua Normalizada (NMI): 4.04
Índice Kappa de Cohen: 17.94


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,60.19,KNeighbors,Entrenamiento
1,Exhaustividad,64.56,KNeighbors,Entrenamiento
2,Puntuación F1,59.44,KNeighbors,Entrenamiento
3,Exactitud,64.56,KNeighbors,Entrenamiento
4,Información Mutua Normalizada (NMI),4.04,KNeighbors,Entrenamiento
5,Índice Kappa de Cohen,17.94,KNeighbors,Entrenamiento


In [19]:
# Predecir las etiquetas para los datos de prueba
resultados_df_knn = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_knn.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_knn_prueba = pd.concat([resultados_df_knn, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_knn_prueba["MODELO"]='KNeighbors'
resultados_df_knn_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_knn_prueba

Precisión:  51.65
Exhaustividad:  58.87
Puntuación F1:  53.6
Exactitud:  58.87
Información Mutua Normalizada (NMI): 0.09
Índice Kappa de Cohen: -0.73


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuación F1,53.60,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
5,Índice Kappa de Cohen,-0.73,KNeighbors,Prueba


In [20]:
mejores_hiperparametros_knn = modelo_knn.get_params()
mejores_hiperparametros_knn

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'manhattan',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 15,
 'p': 2,
 'weights': 'uniform'}

In [21]:
cadena_hiperparametros_knn = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_knn.items()])
df_hiperparametros_knn = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_knn],
    'MODELO': ['KNeighbors'],
    'TIPO_DE_DATOS': ['Entrenamiento']
})
resultados_df_knn = pd.concat([resultados_df_knn_prueba,resultados_df_knn_entrenamiento,df_hiperparametros_knn], ignore_index=True)
resultados_df_knn['TIPO_DE_DATOS']=np.where(resultados_df_knn['MÉTRICA']=='Mejores Hiperparametros','Hiperparametros del modelo',resultados_df_knn['TIPO_DE_DATOS'])
resultados_df_knn

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuación F1,53.6,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Información Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
5,Índice Kappa de Cohen,-0.73,KNeighbors,Prueba
6,Precisión,60.19,KNeighbors,Entrenamiento
7,Exhaustividad,64.56,KNeighbors,Entrenamiento
8,Puntuación F1,59.44,KNeighbors,Entrenamiento
9,Exactitud,64.56,KNeighbors,Entrenamiento


### SVC

In [22]:
estimator = SVC()
estimator.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [23]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':   ['rbf', 'poly', 'sigmoid','linear'], 
                'C': [i/10000 for i in range(8,12,1)],
                'max_iter':[i for i in range(1,3,1)],
                'gamma' : [i/100 for i in range(80,110,5)],
                'random_state':[i for i in range(1,5,1)],
                #'break_ties': [True, False],
                #'cache_size': [100, 200, 300],
                #'class_weight': [None, 'balanced'],
                #'coef0': [0.0, 1.0, 2.0]
                #'decision_function_shape': ['ovo', 'ovr']
                #'degree': [2, 3, 4],
                #'probability': [True, False]
                #'shrinking': [True, False],
                #'tol': [0.0001, 0.001, 0.01],
                #'verbose': [True, False]
                }
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_svc=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_svc,mejores_hiperparametros_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_svc

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.214792299898676
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 1.05, 'kernel': 'sigmoid', 'max_iter': 1, 'random_state': 1}


{'C': 0.0008,
 'gamma': 1.05,
 'kernel': 'sigmoid',
 'max_iter': 1,
 'random_state': 1}

In [24]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_svc.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_entrenamiento = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_entrenamiento["MODELO"]='SVC'
resultados_df_svc_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_svc_entrenamiento

Precisión:  52.76
Exhaustividad:  61.21
Puntuación F1:  49.94
Exactitud:  61.21
Información Mutua Normalizada (NMI): 0.76
Índice Kappa de Cohen: 3.08


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.76,SVC,Entrenamiento
1,Exhaustividad,61.21,SVC,Entrenamiento
2,Puntuación F1,49.94,SVC,Entrenamiento
3,Exactitud,61.21,SVC,Entrenamiento
4,Información Mutua Normalizada (NMI),0.76,SVC,Entrenamiento
5,Índice Kappa de Cohen,3.08,SVC,Entrenamiento


In [25]:
# Predecir las etiquetas para los datos de prueba
resultados_df_SVC = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_svc.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_svc_prueba = pd.concat([resultados_df_SVC, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_svc_prueba["MODELO"]='SVC'
resultados_df_svc_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_svc_prueba

Precisión:  52.51
Exhaustividad:  63.35
Puntuación F1:  52.14
Exactitud:  63.35
Información Mutua Normalizada (NMI): 0.35
Índice Kappa de Cohen: 0.7


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.51,SVC,Prueba
1,Exhaustividad,63.35,SVC,Prueba
2,Puntuación F1,52.14,SVC,Prueba
3,Exactitud,63.35,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.35,SVC,Prueba
5,Índice Kappa de Cohen,0.70,SVC,Prueba


In [26]:
mejores_hiperparametros_svc = modelo_svc.get_params()
mejores_hiperparametros_svc

{'C': 0.0008,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 1.05,
 'kernel': 'sigmoid',
 'max_iter': 1,
 'probability': False,
 'random_state': 1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [27]:
cadena_hiperparametros_svc = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_svc.items()])
df_hiperparametros_svc = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_svc],
    'MODELO': ['SVC'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_svc = pd.concat([resultados_df_svc_prueba,resultados_df_svc_entrenamiento,df_hiperparametros_svc], ignore_index=True)
resultados_df_svc

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.51,SVC,Prueba
1,Exhaustividad,63.35,SVC,Prueba
2,Puntuación F1,52.14,SVC,Prueba
3,Exactitud,63.35,SVC,Prueba
4,Información Mutua Normalizada (NMI),0.35,SVC,Prueba
5,Índice Kappa de Cohen,0.7,SVC,Prueba
6,Precisión,52.76,SVC,Entrenamiento
7,Exhaustividad,61.21,SVC,Entrenamiento
8,Puntuación F1,49.94,SVC,Entrenamiento
9,Exactitud,61.21,SVC,Entrenamiento


### DECISION TREE

In [28]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
              'max_depth':[i for i in range(1,7,1)],
              'min_samples_split' :  [i for i in range(1,7,1)],  
              'min_samples_leaf' : [i for i in range(1,7,1)], 
              'max_features' : [i for i in range(1,7,1)], 
              'splitter': ["best", "random"],
              'random_state': [i for i in range(1,5,1)],
              #'criterion': ['gini','entropy','log_loss']
            
}
    modelo = DecisionTreeClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_tree=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_tree,mejores_hiperparametros_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.91691995947316
Mejor valor PARAMETRO usando k-fold: {'max_depth': 4, 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 3, 'random_state': 4, 'splitter': 'best'}


In [29]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_tree.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_entrenamiento = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_entrenamiento["MODELO"]='DecisionTree'
resultados_df_tree_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_tree_entrenamiento

Precisión:  71.23
Exhaustividad:  64.13
Puntuación F1:  53.64
Exactitud:  64.13
Información Mutua Normalizada (NMI): 4.76
Índice Kappa de Cohen: 10.15


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,71.23,DecisionTree,Entrenamiento
1,Exhaustividad,64.13,DecisionTree,Entrenamiento
2,Puntuación F1,53.64,DecisionTree,Entrenamiento
3,Exactitud,64.13,DecisionTree,Entrenamiento
4,Información Mutua Normalizada (NMI),4.76,DecisionTree,Entrenamiento
5,Índice Kappa de Cohen,10.15,DecisionTree,Entrenamiento


In [30]:
# Predecir las etiquetas para los datos de prueba
resultados_df_tree = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_tree.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_tree_prueba = pd.concat([resultados_df_tree, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_tree_prueba["MODELO"]='DecisionTree'
resultados_df_tree_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_tree_prueba

Precisión:  56.79
Exhaustividad:  64.18
Puntuación F1:  54.02
Exactitud:  64.18
Información Mutua Normalizada (NMI): 0.68
Índice Kappa de Cohen: 3.9


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.79,DecisionTree,Prueba
1,Exhaustividad,64.18,DecisionTree,Prueba
2,Puntuación F1,54.02,DecisionTree,Prueba
3,Exactitud,64.18,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),0.68,DecisionTree,Prueba
5,Índice Kappa de Cohen,3.90,DecisionTree,Prueba


In [31]:
mejores_hiperparametros_tree = modelo_tree.get_params()
mejores_hiperparametros_tree

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 4,
 'max_features': 1,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 4,
 'splitter': 'best'}

In [32]:
cadena_hiperparametros_tree = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_tree.items()])
df_hiperparametros_tree = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_tree],
    'MODELO': ['DecisionTree'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_tree = pd.concat([resultados_df_tree_prueba,resultados_df_tree_entrenamiento,df_hiperparametros_tree], ignore_index=True)
resultados_df_tree

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.79,DecisionTree,Prueba
1,Exhaustividad,64.18,DecisionTree,Prueba
2,Puntuación F1,54.02,DecisionTree,Prueba
3,Exactitud,64.18,DecisionTree,Prueba
4,Información Mutua Normalizada (NMI),0.68,DecisionTree,Prueba
5,Índice Kappa de Cohen,3.9,DecisionTree,Prueba
6,Precisión,71.23,DecisionTree,Entrenamiento
7,Exhaustividad,64.13,DecisionTree,Entrenamiento
8,Puntuación F1,53.64,DecisionTree,Entrenamiento
9,Exactitud,64.13,DecisionTree,Entrenamiento


### NAIVE BAYES

In [33]:
# conocer los hiperparámetro de cada algoritmo
estimator =  GaussianNB()
estimator.get_params()


{'priors': None, 'var_smoothing': 1e-09}

In [34]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo

X_trn = X_trn
Y_trn = Y_trn 

modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 60.993920972644375
Mejor valor PARAMETRO usando k-fold: {}


In [35]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_gaussian.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_entrenamiento = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_entrenamiento["MODELO"]='NaiveBayes'
resultados_df_gaussian_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_entrenamiento

Precisión:  55.74
Exhaustividad:  61.57
Puntuación F1:  49.37
Exactitud:  61.57
Información Mutua Normalizada (NMI): 1.7
Índice Kappa de Cohen: 2.89


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.74,NaiveBayes,Entrenamiento
1,Exhaustividad,61.57,NaiveBayes,Entrenamiento
2,Puntuación F1,49.37,NaiveBayes,Entrenamiento
3,Exactitud,61.57,NaiveBayes,Entrenamiento
4,Información Mutua Normalizada (NMI),1.70,NaiveBayes,Entrenamiento
5,Índice Kappa de Cohen,2.89,NaiveBayes,Entrenamiento


In [36]:
# Predecir las etiquetas para los datos de prueba
resultados_df_gaussian = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_gaussian.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_gaussian_prueba = pd.concat([resultados_df_gaussian, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_gaussian_prueba["MODELO"]='NaiveBayes'
resultados_df_gaussian_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_gaussian_prueba

Precisión:  55.59
Exhaustividad:  64.01
Puntuación F1:  51.91
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.25
Índice Kappa de Cohen: 1.1


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.59,NaiveBayes,Prueba
1,Exhaustividad,64.01,NaiveBayes,Prueba
2,Puntuación F1,51.91,NaiveBayes,Prueba
3,Exactitud,64.01,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),0.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,1.10,NaiveBayes,Prueba


In [37]:
cadena_hiperparametros_gaussian = 'No tiene hiperparametros'
df_hiperparametros_gaussian = pd.DataFrame({
    'MÉTRICA': ['No tiene Hiperparametros'],
    'VALOR': [cadena_hiperparametros_gaussian],
    'MODELO': ['NaiveBayes'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

In [38]:
resultados_df_gaussian = pd.concat([resultados_df_gaussian_prueba,resultados_df_gaussian_entrenamiento,df_hiperparametros_gaussian], ignore_index=True)
resultados_df_gaussian

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.59,NaiveBayes,Prueba
1,Exhaustividad,64.01,NaiveBayes,Prueba
2,Puntuación F1,51.91,NaiveBayes,Prueba
3,Exactitud,64.01,NaiveBayes,Prueba
4,Información Mutua Normalizada (NMI),0.25,NaiveBayes,Prueba
5,Índice Kappa de Cohen,1.1,NaiveBayes,Prueba
6,Precisión,55.74,NaiveBayes,Entrenamiento
7,Exhaustividad,61.57,NaiveBayes,Entrenamiento
8,Puntuación F1,49.37,NaiveBayes,Entrenamiento
9,Exactitud,61.57,NaiveBayes,Entrenamiento


### LDA

In [39]:
# conocer los hiperparámetro de cada algoritmo
estimator = LinearDiscriminantAnalysis()
estimator.get_params()

{'covariance_estimator': None,
 'n_components': None,
 'priors': None,
 'shrinkage': None,
 'solver': 'svd',
 'store_covariance': False,
 'tol': 0.0001}

In [40]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,6,7,8,9,10],
            'shrinkage': ['auto', 0.001, 0.01, 0.1, 0.5,1,10,100,1000],
            'tol':[i/1000 for i in range(1,100,1)]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LDA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo, grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_LDA,mejores_hiperparametros_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_LDA


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.49392097264437
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 1, 'solver': 'lsqr', 'tol': 0.001}


{'n_components': 1, 'shrinkage': 1, 'solver': 'lsqr', 'tol': 0.001}

In [41]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LDA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_entrenamiento = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_entrenamiento["MODELO"]='LDA'
resultados_df_LDA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_entrenamiento

Precisión:  62.58
Exhaustividad:  61.99
Puntuación F1:  48.88
Exactitud:  61.99
Información Mutua Normalizada (NMI): 1.18
Índice Kappa de Cohen: 2.91


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,62.58,LDA,Entrenamiento
1,Exhaustividad,61.99,LDA,Entrenamiento
2,Puntuación F1,48.88,LDA,Entrenamiento
3,Exactitud,61.99,LDA,Entrenamiento
4,Información Mutua Normalizada (NMI),1.18,LDA,Entrenamiento
5,Índice Kappa de Cohen,2.91,LDA,Entrenamiento


In [42]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LDA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LDA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LDA_prueba = pd.concat([resultados_df_LDA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LDA_prueba["MODELO"]='LDA'
resultados_df_LDA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LDA_prueba

Precisión:  52.44
Exhaustividad:  64.18
Puntuación F1:  50.94
Exactitud:  64.18
Información Mutua Normalizada (NMI): 0.44
Índice Kappa de Cohen: 0.41


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.44,LDA,Prueba
1,Exhaustividad,64.18,LDA,Prueba
2,Puntuación F1,50.94,LDA,Prueba
3,Exactitud,64.18,LDA,Prueba
4,Información Mutua Normalizada (NMI),0.44,LDA,Prueba
5,Índice Kappa de Cohen,0.41,LDA,Prueba


In [43]:
mejores_hiperparametros_LDA = modelo_LDA.get_params()
mejores_hiperparametros_LDA

{'covariance_estimator': None,
 'n_components': 1,
 'priors': None,
 'shrinkage': 1,
 'solver': 'lsqr',
 'store_covariance': False,
 'tol': 0.001}

In [44]:
cadena_hiperparametros_LDA = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_LDA.items()])
df_hiperparametros_LDA = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_LDA],
    'MODELO': ['LDA'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_LDA = pd.concat([resultados_df_LDA_prueba,resultados_df_LDA_entrenamiento,df_hiperparametros_LDA], ignore_index=True)
resultados_df_LDA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,52.44,LDA,Prueba
1,Exhaustividad,64.18,LDA,Prueba
2,Puntuación F1,50.94,LDA,Prueba
3,Exactitud,64.18,LDA,Prueba
4,Información Mutua Normalizada (NMI),0.44,LDA,Prueba
5,Índice Kappa de Cohen,0.41,LDA,Prueba
6,Precisión,62.58,LDA,Entrenamiento
7,Exhaustividad,61.99,LDA,Entrenamiento
8,Puntuación F1,48.88,LDA,Entrenamiento
9,Exactitud,61.99,LDA,Entrenamiento


### BAGGINGCLASSIFIER

In [45]:
from sklearn.ensemble import BaggingClassifier
estimator = BaggingClassifier()
estimator.get_params()

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [46]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(70,90,5)],
            'max_features': [i/100.0 for i in range(75,85,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    base_estimator= DecisionTreeClassifier(**mejores_hiperparametros_tree)
    semilla=7
    modelo = BaggingClassifier(estimator=base_estimator,n_estimators=750, random_state=semilla,
                             bootstrap= True, bootstrap_features = True, max_features = 0.7,
                             max_samples= 0.5)
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_BG=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG,mejores_hiperparametros_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.494427558257335
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.75, 'max_samples': 0.85, 'n_estimators': 750}


In [47]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_BG.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_entrenamiento = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_entrenamiento["MODELO"]='Bagging'
resultados_df_BG_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_BG_entrenamiento

Precisión:  97.8
Exhaustividad:  97.72
Puntuación F1:  97.68
Exactitud:  97.72
Información Mutua Normalizada (NMI): 87.6
Índice Kappa de Cohen: 95.43


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,97.80,Bagging,Entrenamiento
1,Exhaustividad,97.72,Bagging,Entrenamiento
2,Puntuación F1,97.68,Bagging,Entrenamiento
3,Exactitud,97.72,Bagging,Entrenamiento
4,Información Mutua Normalizada (NMI),87.60,Bagging,Entrenamiento
5,Índice Kappa de Cohen,95.43,Bagging,Entrenamiento


In [48]:
# Predecir las etiquetas para los datos de prueba
resultados_df_BG = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_BG.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_BG_prueba = pd.concat([resultados_df_BG, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_BG_prueba["MODELO"]='Bagging'
resultados_df_BG_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_BG_prueba

Precisión:  57.09
Exhaustividad:  63.35
Puntuación F1:  56.95
Exactitud:  63.35
Información Mutua Normalizada (NMI): 0.76
Índice Kappa de Cohen: 7.27


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.09,Bagging,Prueba
1,Exhaustividad,63.35,Bagging,Prueba
2,Puntuación F1,56.95,Bagging,Prueba
3,Exactitud,63.35,Bagging,Prueba
4,Información Mutua Normalizada (NMI),0.76,Bagging,Prueba
5,Índice Kappa de Cohen,7.27,Bagging,Prueba


In [49]:
mejores_hiperparametros_BG = modelo_BG.get_params()
mejores_hiperparametros_BG

{'bootstrap': True,
 'bootstrap_features': True,
 'estimator': None,
 'max_features': 0.75,
 'max_samples': 0.85,
 'n_estimators': 750,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [50]:
cadena_hiperparametros_BG = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_BG.items()])
df_hiperparametros_BG = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_BG],
    'MODELO': ['Bagging'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_BG = pd.concat([resultados_df_BG_prueba,resultados_df_BG_entrenamiento,df_hiperparametros_BG], ignore_index=True)
resultados_df_BG

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.09,Bagging,Prueba
1,Exhaustividad,63.35,Bagging,Prueba
2,Puntuación F1,56.95,Bagging,Prueba
3,Exactitud,63.35,Bagging,Prueba
4,Información Mutua Normalizada (NMI),0.76,Bagging,Prueba
5,Índice Kappa de Cohen,7.27,Bagging,Prueba
6,Precisión,97.8,Bagging,Entrenamiento
7,Exhaustividad,97.72,Bagging,Entrenamiento
8,Puntuación F1,97.68,Bagging,Entrenamiento
9,Exactitud,97.72,Bagging,Entrenamiento


### RANDOM FOREST

In [51]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
estimator.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [52]:
def entrenar_modelo_random_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [1, 2 , 3,  4 , 6 , 8 , 10 , 15, 20 ],  
                'min_samples_leaf' : [ 1 , 3 , 5 , 7 , 9, 12, 15 ],
                'criterion':('gini','entropy','log_loss'),
                #'max_features':('sqrt','log2',None)
                #'class_weight':('balanced','balanced_subsample')
              }
    modelo = RandomForestClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_random=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = RandomForestClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_random,mejores_hiperparametros_random = entrenar_modelo_random_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.41742654508611
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'min_samples_leaf': 12, 'min_samples_split': 15}


In [53]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_random.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_entrenamiento = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_entrenamiento["MODELO"]='RandomForest'
resultados_df_random_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_random_entrenamiento

Precisión:  68.94
Exhaustividad:  66.41
Puntuación F1:  57.98
Exactitud:  66.41
Información Mutua Normalizada (NMI): 8.23
Índice Kappa de Cohen: 17.35


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,68.94,RandomForest,Entrenamiento
1,Exhaustividad,66.41,RandomForest,Entrenamiento
2,Puntuación F1,57.98,RandomForest,Entrenamiento
3,Exactitud,66.41,RandomForest,Entrenamiento
4,Información Mutua Normalizada (NMI),8.23,RandomForest,Entrenamiento
5,Índice Kappa de Cohen,17.35,RandomForest,Entrenamiento


In [54]:
# Predecir las etiquetas para los datos de prueba
resultados_df_random = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_random.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_random_prueba = pd.concat([resultados_df_random, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_random_prueba["MODELO"]='RandomForest'
resultados_df_random_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_random_prueba

Precisión:  55.68
Exhaustividad:  63.52
Puntuación F1:  54.45
Exactitud:  63.52
Información Mutua Normalizada (NMI): 0.4
Índice Kappa de Cohen: 3.74


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.68,RandomForest,Prueba
1,Exhaustividad,63.52,RandomForest,Prueba
2,Puntuación F1,54.45,RandomForest,Prueba
3,Exactitud,63.52,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),0.40,RandomForest,Prueba
5,Índice Kappa de Cohen,3.74,RandomForest,Prueba


In [55]:
mejores_hiperparametros_random = modelo_random.get_params()
mejores_hiperparametros_random

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 12,
 'min_samples_split': 15,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [56]:
cadena_hiperparametros_random = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_random.items()])
df_hiperparametros_random = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_random],
    'MODELO': ['RandomForest'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_random = pd.concat([resultados_df_random_prueba,resultados_df_random_entrenamiento,df_hiperparametros_random], ignore_index=True)
resultados_df_random

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.68,RandomForest,Prueba
1,Exhaustividad,63.52,RandomForest,Prueba
2,Puntuación F1,54.45,RandomForest,Prueba
3,Exactitud,63.52,RandomForest,Prueba
4,Información Mutua Normalizada (NMI),0.4,RandomForest,Prueba
5,Índice Kappa de Cohen,3.74,RandomForest,Prueba
6,Precisión,68.94,RandomForest,Entrenamiento
7,Exhaustividad,66.41,RandomForest,Entrenamiento
8,Puntuación F1,57.98,RandomForest,Entrenamiento
9,Exactitud,66.41,RandomForest,Entrenamiento


### EXTRA TREES CLASSIFIER

In [57]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,10,1)], 
                'min_samples_leaf' : [i for i in range(0,10,1)],
                #'max_features':[i for i in range(0,5,1)],
                #'max_depth':[i for i in range(0,5,1)],
                'min_samples_leaf':[i for i in range(0,10,1)],
                'min_samples_split':[i for i in range(0,10,1)],
                'criterion':('gini','entropy','log_loss')}
    
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40,
                                bootstrap=True) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_extra=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_extra,mejores_hiperparametros_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.56129685916918
Mejor valor PARAMETRO usando k-fold: {'criterion': 'gini', 'min_samples_leaf': 2, 'min_samples_split': 5}


In [58]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_extra.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_entrenamiento = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_entrenamiento["MODELO"]='ExtraTrees'
resultados_df_extra_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_extra_entrenamiento


Precisión:  86.4
Exhaustividad:  82.78
Puntuación F1:  80.15
Exactitud:  82.78
Información Mutua Normalizada (NMI): 43.9
Índice Kappa de Cohen: 61.43


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,86.40,ExtraTrees,Entrenamiento
1,Exhaustividad,82.78,ExtraTrees,Entrenamiento
2,Puntuación F1,80.15,ExtraTrees,Entrenamiento
3,Exactitud,82.78,ExtraTrees,Entrenamiento
4,Información Mutua Normalizada (NMI),43.90,ExtraTrees,Entrenamiento
5,Índice Kappa de Cohen,61.43,ExtraTrees,Entrenamiento


In [59]:
# Predecir las etiquetas para los datos de prueba
resultados_df_extra = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_extra.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_extra_prueba = pd.concat([resultados_df_extra, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_extra_prueba["MODELO"]='ExtraTrees'
resultados_df_extra_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_extra_prueba

Precisión:  57.27
Exhaustividad:  64.18
Puntuación F1:  55.49
Exactitud:  64.18
Información Mutua Normalizada (NMI): 1.03
Índice Kappa de Cohen: 5.88


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.27,ExtraTrees,Prueba
1,Exhaustividad,64.18,ExtraTrees,Prueba
2,Puntuación F1,55.49,ExtraTrees,Prueba
3,Exactitud,64.18,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),1.03,ExtraTrees,Prueba
5,Índice Kappa de Cohen,5.88,ExtraTrees,Prueba


In [60]:
mejores_hiperparametros_extra = modelo_extra.get_params()
mejores_hiperparametros_extra

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [61]:
cadena_hiperparametros_extra = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_extra.items()])
df_hiperparametros_extra = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_extra],
    'MODELO': ['ExtraTrees'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_extra = pd.concat([resultados_df_extra_prueba,resultados_df_extra_entrenamiento,df_hiperparametros_extra], ignore_index=True)
resultados_df_extra

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.27,ExtraTrees,Prueba
1,Exhaustividad,64.18,ExtraTrees,Prueba
2,Puntuación F1,55.49,ExtraTrees,Prueba
3,Exactitud,64.18,ExtraTrees,Prueba
4,Información Mutua Normalizada (NMI),1.03,ExtraTrees,Prueba
5,Índice Kappa de Cohen,5.88,ExtraTrees,Prueba
6,Precisión,86.4,ExtraTrees,Entrenamiento
7,Exhaustividad,82.78,ExtraTrees,Entrenamiento
8,Puntuación F1,80.15,ExtraTrees,Entrenamiento
9,Exactitud,82.78,ExtraTrees,Entrenamiento


### ADABOOST

In [62]:
from sklearn.ensemble import AdaBoostClassifier            
estimator  = AdaBoostClassifier ()
estimator.get_params()

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': None}

In [63]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)],
                  'n_estimators':[i for i in range(1,50,1)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_ADA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_ADA, mejores_hiperparametros_ADA= entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.28166160081052
Mejor valor PARAMETRO usando k-fold: {'learning_rate': 0.0015, 'n_estimators': 36}


In [64]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_ADA.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_entrenamiento = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_entrenamiento["MODELO"]='AdaBoost'
resultados_df_ADA_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_entrenamiento

Precisión:  37.55
Exhaustividad:  61.28
Puntuación F1:  46.57
Exactitud:  61.28
Información Mutua Normalizada (NMI): 0.0
Índice Kappa de Cohen: 0.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,37.55,AdaBoost,Entrenamiento
1,Exhaustividad,61.28,AdaBoost,Entrenamiento
2,Puntuación F1,46.57,AdaBoost,Entrenamiento
3,Exactitud,61.28,AdaBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),0.00,AdaBoost,Entrenamiento
5,Índice Kappa de Cohen,0.00,AdaBoost,Entrenamiento


In [65]:
# Predecir las etiquetas para los datos de prueba
resultados_df_ADA = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_ADA.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_ADA_prueba = pd.concat([resultados_df_ADA, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_ADA_prueba["MODELO"]='AdaBoost'
resultados_df_ADA_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_ADA_prueba

Precisión:  41.4
Exhaustividad:  64.34
Puntuación F1:  50.39
Exactitud:  64.34
Información Mutua Normalizada (NMI): 0.0
Índice Kappa de Cohen: 0.0


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,41.40,AdaBoost,Prueba
1,Exhaustividad,64.34,AdaBoost,Prueba
2,Puntuación F1,50.39,AdaBoost,Prueba
3,Exactitud,64.34,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),0.00,AdaBoost,Prueba
5,Índice Kappa de Cohen,0.00,AdaBoost,Prueba


In [66]:
mejores_hiperparametros_ADA = modelo_ADA.get_params()
mejores_hiperparametros_ADA

{'algorithm': 'SAMME.R',
 'estimator': None,
 'learning_rate': 0.0015,
 'n_estimators': 36,
 'random_state': None}

In [67]:
cadena_hiperparametros_ADA = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_ADA.items()])
df_hiperparametros_ADA = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_ADA],
    'MODELO': ['AdaBoost'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_ADA = pd.concat([resultados_df_ADA_prueba,resultados_df_ADA_entrenamiento,df_hiperparametros_ADA], ignore_index=True)
resultados_df_ADA

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,41.4,AdaBoost,Prueba
1,Exhaustividad,64.34,AdaBoost,Prueba
2,Puntuación F1,50.39,AdaBoost,Prueba
3,Exactitud,64.34,AdaBoost,Prueba
4,Información Mutua Normalizada (NMI),0.0,AdaBoost,Prueba
5,Índice Kappa de Cohen,0.0,AdaBoost,Prueba
6,Precisión,37.55,AdaBoost,Entrenamiento
7,Exhaustividad,61.28,AdaBoost,Entrenamiento
8,Puntuación F1,46.57,AdaBoost,Entrenamiento
9,Exactitud,61.28,AdaBoost,Entrenamiento


### GRADIENT BOOSTING MACHINE

In [68]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'learning_rate' : [0.01, 0.05, 0.1,0.15],
                'n_estimators': [i for i in range(100,1200,100)],
                'loss':('log_loss','exponential'),
                'criterion':['friedman_mse']     
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_GD=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD,mejores_hiperparametros_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.77760891590678
Mejor valor PARAMETRO usando k-fold: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'n_estimators': 200}


In [69]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_GD.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_entrenamiento = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_entrenamiento["MODELO"]='GradientBoosting'
resultados_df_GD_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_GD_entrenamiento

Precisión:  72.63
Exhaustividad:  64.34
Puntuación F1:  53.75
Exactitud:  64.34
Información Mutua Normalizada (NMI): 6.72
Índice Kappa de Cohen: 10.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,72.63,GradientBoosting,Entrenamiento
1,Exhaustividad,64.34,GradientBoosting,Entrenamiento
2,Puntuación F1,53.75,GradientBoosting,Entrenamiento
3,Exactitud,64.34,GradientBoosting,Entrenamiento
4,Información Mutua Normalizada (NMI),6.72,GradientBoosting,Entrenamiento
5,Índice Kappa de Cohen,10.66,GradientBoosting,Entrenamiento


In [70]:
# Predecir las etiquetas para los datos de prueba
resultados_df_GD = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_GD.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_GD_prueba = pd.concat([resultados_df_GD, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_GD_prueba["MODELO"]='GradientBoosting'
resultados_df_GD_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_GD_prueba

Precisión:  57.53
Exhaustividad:  64.34
Puntuación F1:  53.39
Exactitud:  64.34
Información Mutua Normalizada (NMI): 0.49
Índice Kappa de Cohen: 3.26


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.53,GradientBoosting,Prueba
1,Exhaustividad,64.34,GradientBoosting,Prueba
2,Puntuación F1,53.39,GradientBoosting,Prueba
3,Exactitud,64.34,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),0.49,GradientBoosting,Prueba
5,Índice Kappa de Cohen,3.26,GradientBoosting,Prueba


In [71]:
mejores_hiperparametros_GD = modelo_GD.get_params()
mejores_hiperparametros_GD

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.01,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [72]:
cadena_hiperparametros_GD = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_GD.items()])
df_hiperparametros_GD = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_GD],
    'MODELO': ['GradientBoosting'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_GD = pd.concat([resultados_df_GD_prueba,resultados_df_GD_entrenamiento,df_hiperparametros_GD], ignore_index=True)
resultados_df_GD

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.53,GradientBoosting,Prueba
1,Exhaustividad,64.34,GradientBoosting,Prueba
2,Puntuación F1,53.39,GradientBoosting,Prueba
3,Exactitud,64.34,GradientBoosting,Prueba
4,Información Mutua Normalizada (NMI),0.49,GradientBoosting,Prueba
5,Índice Kappa de Cohen,3.26,GradientBoosting,Prueba
6,Precisión,72.63,GradientBoosting,Entrenamiento
7,Exhaustividad,64.34,GradientBoosting,Entrenamiento
8,Puntuación F1,53.75,GradientBoosting,Entrenamiento
9,Exactitud,64.34,GradientBoosting,Entrenamiento


### XGBOOST

In [73]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/1000.0 for i in range(100,150,5)],
                'n_estimators':  [i for i in range(1,10,2)],
                'colsample_bytree': [0.1,0.3, 0.5,0.6,0.7,0.8, 0.9, 1,1.1],
                'objective' : ('binary:logistic', 'Multi: softprob'),
                'loss': ['log_loss'],
                'max_features':('sqrt','log2')
                }
    semilla=7
    modelo = XGBClassifier(random_state=semilla,subsample =1,max_depth =2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_XB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_XB,mejores_hiperparametros_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.2046605876393
Mejor valor PARAMETRO usando k-fold: {'colsample_bytree': 0.9, 'loss': 'log_loss', 'max_features': 'sqrt', 'n_estimators': 5, 'objective': 'binary:logistic', 'reg_alpha': 0.2, 'reg_lambda': 0.12}


In [74]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_XB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_entrenamiento = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_entrenamiento["MODELO"]='XGB'
resultados_df_XB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_XB_entrenamiento

Precisión:  77.84
Exhaustividad:  71.1
Puntuación F1:  65.68
Exactitud:  71.1
Información Mutua Normalizada (NMI): 18.96
Índice Kappa de Cohen: 31.23


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,77.84,XGB,Entrenamiento
1,Exhaustividad,71.10,XGB,Entrenamiento
2,Puntuación F1,65.68,XGB,Entrenamiento
3,Exactitud,71.10,XGB,Entrenamiento
4,Información Mutua Normalizada (NMI),18.96,XGB,Entrenamiento
5,Índice Kappa de Cohen,31.23,XGB,Entrenamiento


In [75]:
# Predecir las etiquetas para los datos de prueba
resultados_df_XB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_XB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_XB_prueba = pd.concat([resultados_df_XB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_XB_prueba["MODELO"]='XGB'
resultados_df_XB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_XB_prueba

Precisión:  53.77
Exhaustividad:  61.86
Puntuación F1:  54.38
Exactitud:  61.86
Información Mutua Normalizada (NMI): 0.35
Índice Kappa de Cohen: 2.32


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.77,XGB,Prueba
1,Exhaustividad,61.86,XGB,Prueba
2,Puntuación F1,54.38,XGB,Prueba
3,Exactitud,61.86,XGB,Prueba
4,Información Mutua Normalizada (NMI),0.35,XGB,Prueba
5,Índice Kappa de Cohen,2.32,XGB,Prueba


In [76]:
mejores_hiperparametros_XB = modelo_XB.get_params()
mejores_hiperparametros_XB

{'objective': 'multi:softprob',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.9,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 5,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 0.2,
 'reg_lambda': 0.12,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'loss': 'log_loss',
 'max_features': 'sqrt'}

In [77]:
cadena_hiperparametros_XB = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_XB.items()])
df_hiperparametros_XB = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_XB],
    'MODELO': ['XGB'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_XB = pd.concat([resultados_df_XB_prueba,resultados_df_XB_entrenamiento,df_hiperparametros_XB], ignore_index=True)
resultados_df_XB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.77,XGB,Prueba
1,Exhaustividad,61.86,XGB,Prueba
2,Puntuación F1,54.38,XGB,Prueba
3,Exactitud,61.86,XGB,Prueba
4,Información Mutua Normalizada (NMI),0.35,XGB,Prueba
5,Índice Kappa de Cohen,2.32,XGB,Prueba
6,Precisión,77.84,XGB,Entrenamiento
7,Exhaustividad,71.1,XGB,Entrenamiento
8,Puntuación F1,65.68,XGB,Entrenamiento
9,Exactitud,71.1,XGB,Entrenamiento


### CATBOOST

In [78]:
estimator  = CatBoostClassifier ()
estimator.get_params()

{}

In [79]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42],'learning_rate': [0.01],
                  'depth': [4, 6, 8],'thread_count': [4, 8, 12]
                  } 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_CB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_CB,mejores_hiperparametros_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 61.91995947315097
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'depth': 6, 'l2_leaf_reg': 42, 'learning_rate': 0.01, 'thread_count': 4}


In [80]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_CB.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento.ravel())
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_entrenamiento = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_entrenamiento["MODELO"]='CatBoost'
resultados_df_CB_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_CB_entrenamiento

Precisión:  64.96
Exhaustividad:  63.49
Puntuación F1:  52.47
Exactitud:  63.49
Información Mutua Normalizada (NMI): 3.3
Índice Kappa de Cohen: 8.19


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,64.96,CatBoost,Entrenamiento
1,Exhaustividad,63.49,CatBoost,Entrenamiento
2,Puntuación F1,52.47,CatBoost,Entrenamiento
3,Exactitud,63.49,CatBoost,Entrenamiento
4,Información Mutua Normalizada (NMI),3.30,CatBoost,Entrenamiento
5,Índice Kappa de Cohen,8.19,CatBoost,Entrenamiento


In [81]:
# Predecir las etiquetas para los datos de prueba
resultados_df_CB = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_CB.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba.ravel())
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_CB_prueba = pd.concat([resultados_df_CB, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_CB_prueba["MODELO"]='CatBoost'
resultados_df_CB_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_CB_prueba

Precisión:  58.28
Exhaustividad:  64.51
Puntuación F1:  53.48
Exactitud:  64.51
Información Mutua Normalizada (NMI): 0.62
Índice Kappa de Cohen: 3.57


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,58.28,CatBoost,Prueba
1,Exhaustividad,64.51,CatBoost,Prueba
2,Puntuación F1,53.48,CatBoost,Prueba
3,Exactitud,64.51,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),0.62,CatBoost,Prueba
5,Índice Kappa de Cohen,3.57,CatBoost,Prueba


In [82]:
mejores_hiperparametros_CB = modelo_CB.get_params()
mejores_hiperparametros_CB

{'learning_rate': 0.01,
 'depth': 6,
 'l2_leaf_reg': 42,
 'border_count': 53,
 'thread_count': 4,
 'verbose': 0}

In [83]:
cadena_hiperparametros_CB = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_CB.items()])
df_hiperparametros_CB = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_CB],
    'MODELO': ['CatBoost'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_CB = pd.concat([resultados_df_CB_prueba,resultados_df_CB_entrenamiento,df_hiperparametros_CB], ignore_index=True)
resultados_df_CB

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,58.28,CatBoost,Prueba
1,Exhaustividad,64.51,CatBoost,Prueba
2,Puntuación F1,53.48,CatBoost,Prueba
3,Exactitud,64.51,CatBoost,Prueba
4,Información Mutua Normalizada (NMI),0.62,CatBoost,Prueba
5,Índice Kappa de Cohen,3.57,CatBoost,Prueba
6,Precisión,64.96,CatBoost,Entrenamiento
7,Exhaustividad,63.49,CatBoost,Entrenamiento
8,Puntuación F1,52.47,CatBoost,Entrenamiento
9,Exactitud,63.49,CatBoost,Entrenamiento


### LIGHT GBM

In [84]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {
    'min_child_samples' : [i for i in range(200, 10000, 100)],'colsample_bytree': [0.6, 0.8, 1.0],
    'boosting_type': ['gbdt', 'dart', 'goss'],'objective': ['binary', 'multiclass'],'random_state': [42]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                            num_leaves =  10,max_depth = 1, n_estimators = 100,    
                            learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                            colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                            min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LIGHT=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_LIGHT,mejores_hiperparametros_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 1405, number of used features: 6
[LightGBM] [Info] Start training from score -1.080276
[LightGBM] [Info] Start training from score -0.489698
[LightGBM] [Info] Start training from score -3.058138
[LightGBM] [Info] Start training from score -7.247793
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 62.06484295845998
Mejor valor PARAMETRO usando k-fold: {'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'min_child_samples': 200, 'objective': 'multiclass', 'random_state': 42}


In [85]:
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_LIGHT.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_entrenamiento = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_entrenamiento["MODELO"]='LGBM'
resultados_df_LIGHT_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_entrenamiento

Precisión:  65.48
Exhaustividad:  65.48
Puntuación F1:  57.21
Exactitud:  65.48
Información Mutua Normalizada (NMI): 6.86
Índice Kappa de Cohen: 15.56


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,65.48,LGBM,Entrenamiento
1,Exhaustividad,65.48,LGBM,Entrenamiento
2,Puntuación F1,57.21,LGBM,Entrenamiento
3,Exactitud,65.48,LGBM,Entrenamiento
4,Información Mutua Normalizada (NMI),6.86,LGBM,Entrenamiento
5,Índice Kappa de Cohen,15.56,LGBM,Entrenamiento


In [86]:
# Predecir las etiquetas para los datos de prueba
resultados_df_LIGHT = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_LIGHT.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_LIGHT_prueba = pd.concat([resultados_df_LIGHT, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_LIGHT_prueba["MODELO"]='LGBM'
resultados_df_LIGHT_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_LIGHT_prueba

Precisión:  56.13
Exhaustividad:  63.35
Puntuación F1:  55.09
Exactitud:  63.35
Información Mutua Normalizada (NMI): 0.42
Índice Kappa de Cohen: 4.36


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.13,LGBM,Prueba
1,Exhaustividad,63.35,LGBM,Prueba
2,Puntuación F1,55.09,LGBM,Prueba
3,Exactitud,63.35,LGBM,Prueba
4,Información Mutua Normalizada (NMI),0.42,LGBM,Prueba
5,Índice Kappa de Cohen,4.36,LGBM,Prueba


In [87]:
mejores_hiperparametros_LIGHT = modelo_LIGHT.get_params()
mejores_hiperparametros_LIGHT

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.8,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 200,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': 42,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'verbose': -1}

In [88]:
cadena_hiperparametros_LIGHT = ', '.join([f"{key}: {value}" for key, value in mejores_hiperparametros_LIGHT.items()])
df_hiperparametros_LIGHT = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_LIGHT],
    'MODELO': ['LGBM'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_LIGHT = pd.concat([resultados_df_LIGHT_prueba,resultados_df_LIGHT_entrenamiento,df_hiperparametros_LIGHT], ignore_index=True)
resultados_df_LIGHT

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.13,LGBM,Prueba
1,Exhaustividad,63.35,LGBM,Prueba
2,Puntuación F1,55.09,LGBM,Prueba
3,Exactitud,63.35,LGBM,Prueba
4,Información Mutua Normalizada (NMI),0.42,LGBM,Prueba
5,Índice Kappa de Cohen,4.36,LGBM,Prueba
6,Precisión,65.48,LGBM,Entrenamiento
7,Exhaustividad,65.48,LGBM,Entrenamiento
8,Puntuación F1,57.21,LGBM,Entrenamiento
9,Exactitud,65.48,LGBM,Entrenamiento


### VOTING - VOTACIÓN DURA

In [89]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo7 = XGBClassifier(**mejores_hiperparametros_XB)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    mejores_hiperparametros_voting_hard=mejor_modelo.get_params
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo, mejores_hiperparametros_voting_hard
modelo_voting_hard, mejores_hiperparametros_voting_hard= entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.77710233029381  % 


In [90]:
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_hard.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_entrenamiento = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_entrenamiento["MODELO"]='VotingHard'
resultados_df_voting_hard_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_entrenamiento

Precisión:  73.08
Exhaustividad:  67.12
Puntuación F1:  58.36
Exactitud:  67.12
Información Mutua Normalizada (NMI): 11.92
Índice Kappa de Cohen: 18.55


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,73.08,VotingHard,Entrenamiento
1,Exhaustividad,67.12,VotingHard,Entrenamiento
2,Puntuación F1,58.36,VotingHard,Entrenamiento
3,Exactitud,67.12,VotingHard,Entrenamiento
4,Información Mutua Normalizada (NMI),11.92,VotingHard,Entrenamiento
5,Índice Kappa de Cohen,18.55,VotingHard,Entrenamiento


In [91]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_hard = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_hard.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_hard_prueba = pd.concat([resultados_df_voting_hard, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_hard_prueba["MODELO"]='VotingHard'
resultados_df_voting_hard_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_hard_prueba

Precisión:  57.78
Exhaustividad:  64.34
Puntuación F1:  54.08
Exactitud:  64.34
Información Mutua Normalizada (NMI): 0.58
Índice Kappa de Cohen: 4.08


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.78,VotingHard,Prueba
1,Exhaustividad,64.34,VotingHard,Prueba
2,Puntuación F1,54.08,VotingHard,Prueba
3,Exactitud,64.34,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),0.58,VotingHard,Prueba
5,Índice Kappa de Cohen,4.08,VotingHard,Prueba


In [92]:
estimadores = modelo_voting_hard.estimators
hiperparametros_voting_hard = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_voting_hard.append(hiperparametros_estimador)

hiperparametros_voting_hard

[{'Estimador': 'Gradient',
  'Hiperparametros': {'ccp_alpha': 0.0,
   'criterion': 'friedman_mse',
   'init': None,
   'learning_rate': 0.01,
   'loss': 'log_loss',
   'max_depth': 3,
   'max_features': None,
   'max_leaf_nodes': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 200,
   'n_iter_no_change': None,
   'random_state': None,
   'subsample': 1.0,
   'tol': 0.0001,
   'validation_fraction': 0.1,
   'verbose': 0,
   'warm_start': False}},
 {'Estimador': 'Adaboost',
  'Hiperparametros': {'algorithm': 'SAMME.R',
   'estimator': None,
   'learning_rate': 0.0015,
   'n_estimators': 36,
   'random_state': None}},
 {'Estimador': 'Extratrees',
  'Hiperparametros': {'bootstrap': False,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   '

In [93]:
cadena_hiperparametros_voting_hard = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_voting_hard])

df_hiperparametros_voting_hard = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_voting_hard],
    'MODELO': ['VotingHard'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})

resultados_df_voting_hard = pd.concat([resultados_df_voting_hard_prueba,resultados_df_voting_hard_entrenamiento,df_hiperparametros_voting_hard], ignore_index=True)
resultados_df_voting_hard

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.78,VotingHard,Prueba
1,Exhaustividad,64.34,VotingHard,Prueba
2,Puntuación F1,54.08,VotingHard,Prueba
3,Exactitud,64.34,VotingHard,Prueba
4,Información Mutua Normalizada (NMI),0.58,VotingHard,Prueba
5,Índice Kappa de Cohen,4.08,VotingHard,Prueba
6,Precisión,73.08,VotingHard,Entrenamiento
7,Exhaustividad,67.12,VotingHard,Entrenamiento
8,Puntuación F1,58.36,VotingHard,Entrenamiento
9,Exactitud,67.12,VotingHard,Entrenamiento


### VOTING - VOTACIÓN SUAVE

In [94]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_voting_soft=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_voting_soft

# Cargar los datos
#datos = cargar_datos(carrera, semestre)
X_trn = X_trn
Y_trn = Y_trn 

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft,mejores_hiperparametros_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 62.6322188449848  % 


In [95]:
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_voting_soft.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_entrenamiento = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_entrenamiento["MODELO"]='VotingSoft'
resultados_df_voting_soft_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_entrenamiento

Precisión:  73.74
Exhaustividad:  69.32
Puntuación F1:  62.2
Exactitud:  69.32
Información Mutua Normalizada (NMI): 15.45
Índice Kappa de Cohen: 25.28


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,73.74,VotingSoft,Entrenamiento
1,Exhaustividad,69.32,VotingSoft,Entrenamiento
2,Puntuación F1,62.20,VotingSoft,Entrenamiento
3,Exactitud,69.32,VotingSoft,Entrenamiento
4,Información Mutua Normalizada (NMI),15.45,VotingSoft,Entrenamiento
5,Índice Kappa de Cohen,25.28,VotingSoft,Entrenamiento


In [96]:
# Predecir las etiquetas para los datos de prueba
resultados_df_voting_soft = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_voting_soft.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_voting_soft_prueba = pd.concat([resultados_df_voting_soft, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_voting_soft_prueba["MODELO"]='VotingSoft'
resultados_df_voting_soft_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_voting_soft_prueba

Precisión:  56.11
Exhaustividad:  64.01
Puntuación F1:  53.69
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.57
Índice Kappa de Cohen: 3.32


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.11,VotingSoft,Prueba
1,Exhaustividad,64.01,VotingSoft,Prueba
2,Puntuación F1,53.69,VotingSoft,Prueba
3,Exactitud,64.01,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),0.57,VotingSoft,Prueba
5,Índice Kappa de Cohen,3.32,VotingSoft,Prueba


In [97]:
estimadores = modelo_voting_soft.estimators
hiperparametros_voting_soft = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_voting_soft.append(hiperparametros_estimador)



In [98]:
cadena_hiperparametros_voting_soft = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_voting_soft])

df_hiperparametros_voting_soft = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_voting_soft],
    'MODELO': ['VotingSoft'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_voting_soft = pd.concat([resultados_df_voting_soft_prueba,resultados_df_voting_soft_entrenamiento,df_hiperparametros_voting_soft], ignore_index=True)
resultados_df_voting_soft

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.11,VotingSoft,Prueba
1,Exhaustividad,64.01,VotingSoft,Prueba
2,Puntuación F1,53.69,VotingSoft,Prueba
3,Exactitud,64.01,VotingSoft,Prueba
4,Información Mutua Normalizada (NMI),0.57,VotingSoft,Prueba
5,Índice Kappa de Cohen,3.32,VotingSoft,Prueba
6,Precisión,73.74,VotingSoft,Entrenamiento
7,Exhaustividad,69.32,VotingSoft,Entrenamiento
8,Puntuación F1,62.2,VotingSoft,Entrenamiento
9,Exactitud,69.32,VotingSoft,Entrenamiento


### STACKING ( METAMODELO LINEAL)

In [99]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_lineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_lineal

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal,mejores_hiperparametros_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 61.63627152988854  % 


In [100]:
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_lineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_entrenamiento = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_entrenamiento["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_entrenamiento

Precisión:  78.55
Exhaustividad:  77.08
Puntuación F1:  73.19
Exactitud:  77.08
Información Mutua Normalizada (NMI): 32.24
Índice Kappa de Cohen: 46.75


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,78.55,StackingLineal,Entrenamiento
1,Exhaustividad,77.08,StackingLineal,Entrenamiento
2,Puntuación F1,73.19,StackingLineal,Entrenamiento
3,Exactitud,77.08,StackingLineal,Entrenamiento
4,Información Mutua Normalizada (NMI),32.24,StackingLineal,Entrenamiento
5,Índice Kappa de Cohen,46.75,StackingLineal,Entrenamiento


In [101]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_lineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_stacking_lineal.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_lineal_prueba = pd.concat([resultados_df_stacking_lineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_lineal_prueba["MODELO"]='StackingLineal'
resultados_df_stacking_lineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_lineal_prueba

Precisión:  58.49
Exhaustividad:  64.68
Puntuación F1:  54.31
Exactitud:  64.68
Información Mutua Normalizada (NMI): 1.07
Índice Kappa de Cohen: 4.84


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,58.49,StackingLineal,Prueba
1,Exhaustividad,64.68,StackingLineal,Prueba
2,Puntuación F1,54.31,StackingLineal,Prueba
3,Exactitud,64.68,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),1.07,StackingLineal,Prueba
5,Índice Kappa de Cohen,4.84,StackingLineal,Prueba


In [102]:
estimadores = modelo_stacking_lineal.estimators
hiperparametros_stacking_lineal = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_stacking_lineal.append(hiperparametros_estimador)



In [103]:
cadena_hiperparametros_stacking_lineal = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_stacking_lineal])

df_hiperparametros_stacking_lineal = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_stacking_lineal],
    'MODELO': ['StackingLineal'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})


resultados_df_stacking_lineal = pd.concat([resultados_df_stacking_lineal_prueba,resultados_df_stacking_lineal_entrenamiento,df_hiperparametros_stacking_lineal], ignore_index=True)
resultados_df_stacking_lineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,58.49,StackingLineal,Prueba
1,Exhaustividad,64.68,StackingLineal,Prueba
2,Puntuación F1,54.31,StackingLineal,Prueba
3,Exactitud,64.68,StackingLineal,Prueba
4,Información Mutua Normalizada (NMI),1.07,StackingLineal,Prueba
5,Índice Kappa de Cohen,4.84,StackingLineal,Prueba
6,Precisión,78.55,StackingLineal,Entrenamiento
7,Exhaustividad,77.08,StackingLineal,Entrenamiento
8,Puntuación F1,73.19,StackingLineal,Entrenamiento
9,Exactitud,77.08,StackingLineal,Entrenamiento


### STACKING (METAMODELO NO LINEAL)

In [104]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo1),('Random Forest',modelo2),('Bagging',modelo3),
                                    ('Decision tree',modelo4)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_nolineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_nolineal
X_trn = X_trn
Y_trn = Y_trn 
modelo_stacking_nolineal,mejores_hiperparametros_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 60.28267477203647  % 


In [105]:
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_stacking_nolineal.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_entrenamiento = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_entrenamiento["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_entrenamiento

Precisión:  79.19
Exhaustividad:  82.7
Puntuación F1:  80.43
Exactitud:  82.7
Información Mutua Normalizada (NMI): 37.31
Índice Kappa de Cohen: 62.82


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,79.19,Stackingnolineal,Entrenamiento
1,Exhaustividad,82.70,Stackingnolineal,Entrenamiento
2,Puntuación F1,80.43,Stackingnolineal,Entrenamiento
3,Exactitud,82.70,Stackingnolineal,Entrenamiento
4,Información Mutua Normalizada (NMI),37.31,Stackingnolineal,Entrenamiento
5,Índice Kappa de Cohen,62.82,Stackingnolineal,Entrenamiento


In [106]:
# Predecir las etiquetas para los datos de prueba
resultados_df_stacking_nolineal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])
Y_pred_prueba = modelo_stacking_nolineal.predict(X_tst)
precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_stacking_nolineal_prueba = pd.concat([resultados_df_stacking_nolineal, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_stacking_nolineal_prueba["MODELO"]='Stackingnolineal'
resultados_df_stacking_nolineal_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_stacking_nolineal_prueba

Precisión:  55.74
Exhaustividad:  62.85
Puntuación F1:  55.66
Exactitud:  62.85
Información Mutua Normalizada (NMI): 0.38
Índice Kappa de Cohen: 4.88


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.74,Stackingnolineal,Prueba
1,Exhaustividad,62.85,Stackingnolineal,Prueba
2,Puntuación F1,55.66,Stackingnolineal,Prueba
3,Exactitud,62.85,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),0.38,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,4.88,Stackingnolineal,Prueba


In [107]:
estimadores = modelo_stacking_nolineal.estimators
hiperparametros_stacking_nolineal = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_stacking_nolineal.append(hiperparametros_estimador)


In [108]:
cadena_hiperparametros_stacking_nolineal = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_stacking_nolineal])

df_hiperparametros_stacking_nolineal = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_stacking_nolineal],
    'MODELO': ['Stackingnolineal'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_stacking_nolineal = pd.concat([resultados_df_stacking_nolineal_prueba,resultados_df_stacking_nolineal_entrenamiento,df_hiperparametros_stacking_nolineal], ignore_index=True)
resultados_df_stacking_nolineal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,55.74,Stackingnolineal,Prueba
1,Exhaustividad,62.85,Stackingnolineal,Prueba
2,Puntuación F1,55.66,Stackingnolineal,Prueba
3,Exactitud,62.85,Stackingnolineal,Prueba
4,Información Mutua Normalizada (NMI),0.38,Stackingnolineal,Prueba
5,Índice Kappa de Cohen,4.88,Stackingnolineal,Prueba
6,Precisión,79.19,Stackingnolineal,Entrenamiento
7,Exhaustividad,82.7,Stackingnolineal,Entrenamiento
8,Puntuación F1,80.43,Stackingnolineal,Entrenamiento
9,Exactitud,82.7,Stackingnolineal,Entrenamiento


### VOTING WEIGHTED VOTING 

In [109]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

def weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    modelos = [
        GradientBoostingClassifier(**mejores_hiperparametros_GD),
        AdaBoostClassifier(**mejores_hiperparametros_ADA),
        ExtraTreesClassifier(**mejores_hiperparametros_extra),
        RandomForestClassifier(**mejores_hiperparametros_random),
        BaggingClassifier(**mejores_hiperparametros_BG),
        DecisionTreeClassifier(**mejores_hiperparametros_tree),
        XGBClassifier(**mejores_hiperparametros_XB)
    ]
    pesos = np.ones(len(modelos))
    precisión_modelos = []
    for modelo in modelos:
        modelo.fit(X_trn_transformado, Y_trn)
        precisión = np.mean(cross_val_score(modelo, X_trn_transformado, Y_trn, cv=KFold(n_splits=10, random_state=7, shuffle=True), scoring='accuracy'))
        precisión_modelos.append(precisión)
    suma_precisión = sum(precisión_modelos)
    for i in range(len(precisión_modelos)):
        pesos[i] = suma_precisión / (precisión_modelos[i] * len(modelos))

    return modelos, pesos

def weighted_voting_predict(modelos, pesos, X_test):
    predicciones_modelos = [modelo.predict(X_test) for modelo in modelos]
    pesos_float64 = np.array(pesos, dtype=np.float64)
    predicciones_ponderadas = np.zeros_like(predicciones_modelos[0], dtype=np.float64)
    for i, predicciones_modelo in enumerate(predicciones_modelos):
        predicciones_ponderadas += predicciones_modelo * pesos_float64[i]
    predicciones_ponderadas /= len(modelos)
    predicciones_ponderadas = np.round(predicciones_ponderadas).astype(int)
    
    return predicciones_ponderadas

# Utilizar la función
modelos, pesos = weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB)
Y_pred = weighted_voting_predict(modelos, pesos, X_tst)
precision = accuracy_score(Y_tst, Y_pred)
recall = recall_score(Y_tst, Y_pred, average='weighted')
f1 = f1_score(Y_tst, Y_pred, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred)
print("Precisión: ", precision)
print("Exhaustividad (Recall): ", recall)
print("Puntuación F1: ", f1)
print("Exactitud: ", accuracy)
print("Precisión del modelo (Ensamblaje con voto ponderado): {:.2f}%".format(precision * 100))


Precisión:  0.6401326699834162
Exhaustividad (Recall):  0.6401326699834162
Puntuación F1:  0.5342989065736325
Exactitud:  0.6401326699834162
Precisión del modelo (Ensamblaje con voto ponderado): 64.01%


In [110]:
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= weighted_voting_predict(modelos, pesos, X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_entrenamiento = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_entrenamiento["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_entrenamiento

Precisión:  73.37
Exhaustividad:  66.98
Puntuación F1:  58.1
Exactitud:  66.98
Información Mutua Normalizada (NMI): 11.99
Índice Kappa de Cohen: 18.08


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,73.37,VotingWeighted,Entrenamiento
1,Exhaustividad,66.98,VotingWeighted,Entrenamiento
2,Puntuación F1,58.10,VotingWeighted,Entrenamiento
3,Exactitud,66.98,VotingWeighted,Entrenamiento
4,Información Mutua Normalizada (NMI),11.99,VotingWeighted,Entrenamiento
5,Índice Kappa de Cohen,18.08,VotingWeighted,Entrenamiento


In [111]:
# Predecir las etiquetas para los datos de prueba
resultados_df_weighted_voting = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba= weighted_voting_predict(modelos, pesos, X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_weighted_voting_prueba = pd.concat([resultados_df_weighted_voting, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_weighted_voting_prueba["MODELO"]='VotingWeighted'
resultados_df_weighted_voting_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_weighted_voting_prueba

Precisión:  56.37
Exhaustividad:  64.01
Puntuación F1:  53.43
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.33
Índice Kappa de Cohen: 2.91


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.37,VotingWeighted,Prueba
1,Exhaustividad,64.01,VotingWeighted,Prueba
2,Puntuación F1,53.43,VotingWeighted,Prueba
3,Exactitud,64.01,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),0.33,VotingWeighted,Prueba
5,Índice Kappa de Cohen,2.91,VotingWeighted,Prueba


In [112]:
resultados_df_weighted_voting = pd.concat([resultados_df_weighted_voting_prueba,resultados_df_weighted_voting_entrenamiento], ignore_index=True)
resultados_df_weighted_voting

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,56.37,VotingWeighted,Prueba
1,Exhaustividad,64.01,VotingWeighted,Prueba
2,Puntuación F1,53.43,VotingWeighted,Prueba
3,Exactitud,64.01,VotingWeighted,Prueba
4,Información Mutua Normalizada (NMI),0.33,VotingWeighted,Prueba
5,Índice Kappa de Cohen,2.91,VotingWeighted,Prueba
6,Precisión,73.37,VotingWeighted,Entrenamiento
7,Exhaustividad,66.98,VotingWeighted,Entrenamiento
8,Puntuación F1,58.10,VotingWeighted,Entrenamiento
9,Exactitud,66.98,VotingWeighted,Entrenamiento


### SUPER APRENDIZ MLENS

In [113]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random,mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = GradientBoostingClassifier(**mejores_hiperparametros_GD) 
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2), 
                ('Bagging', modelo3), ('Decision tree', modelo4),('Gradient',modelo5)]
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    super_learner.add_meta(estimador_final)
    super_learner.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_super_learner=super_learner.get_params
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner,mejores_hiperparametros_super_learner,estimadores
modelo_superaprendiz,mejores_hiperparametros_super_learner,estimadores = entrenar_modelo_super_aprendiz(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random,mejores_hiperparametros_BG)


Fitting 2 layers


Processing layer-1             done | 00:00:20
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:20

Fitting 2 layers
Processing layer-1             done | 00:00:20
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:20

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:23
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:24

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:30
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:30

Predicting 2 layers
Processing layer-1             done |

In [114]:
print(modelo_superaprendiz.data)

                           ft-m  ft-s  pt-m  pt-s
layer-1  bagging          12.60  0.11  1.13  0.10
layer-1  decision-tree     0.02  0.01  0.00  0.01
layer-1  extratrees        1.34  0.12  0.18  0.03
layer-1  gradient         15.42  4.46  0.00  0.00
layer-1  random-forest     1.02  0.21  0.08  0.02



In [115]:
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_entrenamiento = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_entrenamiento["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_entrenamiento


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  80.98
Exhaustividad:  82.35
Puntuación F1:  79.65
Exactitud:  82.35
Información Mutua Normalizada (NMI): 41.09
Índice Kappa de Cohen: 60.56


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,80.98,SuperAprendiz,Entrenamiento
1,Exhaustividad,82.35,SuperAprendiz,Entrenamiento
2,Puntuación F1,79.65,SuperAprendiz,Entrenamiento
3,Exactitud,82.35,SuperAprendiz,Entrenamiento
4,Información Mutua Normalizada (NMI),41.09,SuperAprendiz,Entrenamiento
5,Índice Kappa de Cohen,60.56,SuperAprendiz,Entrenamiento


In [116]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_prueba = pd.concat([resultados_df_superaprendiz, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_prueba["MODELO"]='SuperAprendiz'
resultados_df_superaprendiz_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_prueba


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  57.06
Exhaustividad:  64.18
Puntuación F1:  55.1
Exactitud:  64.18
Información Mutua Normalizada (NMI): 1.03
Índice Kappa de Cohen: 5.36


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.06,SuperAprendiz,Prueba
1,Exhaustividad,64.18,SuperAprendiz,Prueba
2,Puntuación F1,55.10,SuperAprendiz,Prueba
3,Exactitud,64.18,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),1.03,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,5.36,SuperAprendiz,Prueba


In [117]:
hiperparametros_superaprendiz = []
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_superaprendiz.append(hiperparametros_estimador)


In [118]:
cadena_hiperparametros_superaprendiz = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_superaprendiz])

df_hiperparametros_superaprendiz = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_superaprendiz],
    'MODELO': ['SuperAprendiz'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_superaprendiz = pd.concat([resultados_df_superaprendiz_prueba,resultados_df_superaprendiz_entrenamiento,df_hiperparametros_superaprendiz], ignore_index=True)
resultados_df_superaprendiz

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.06,SuperAprendiz,Prueba
1,Exhaustividad,64.18,SuperAprendiz,Prueba
2,Puntuación F1,55.1,SuperAprendiz,Prueba
3,Exactitud,64.18,SuperAprendiz,Prueba
4,Información Mutua Normalizada (NMI),1.03,SuperAprendiz,Prueba
5,Índice Kappa de Cohen,5.36,SuperAprendiz,Prueba
6,Precisión,80.98,SuperAprendiz,Entrenamiento
7,Exhaustividad,82.35,SuperAprendiz,Entrenamiento
8,Puntuación F1,79.65,SuperAprendiz,Entrenamiento
9,Exactitud,82.35,SuperAprendiz,Entrenamiento


### SUPER APRENDIZ MULTICAPA

In [119]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                                            mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2), 
                ('Bagging', modelo3), ('Decision tree', modelo4)]
    superaprendiz_dos_capas = SuperLearner(folds=10, random_state=semilla, verbose=2)
    superaprendiz_dos_capas.add(estimadores)
    superaprendiz_dos_capas.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    superaprendiz_dos_capas.add_meta(estimador_final)
    
    superaprendiz_dos_capas.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_superaprendiz_dos_capas=superaprendiz_dos_capas.get_params
    resultados = cross_val_score(superaprendiz_dos_capas, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores
modelo_superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores = entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random,mejores_hiperparametros_BG)


Fitting 3 layers


Processing layer-1             done | 00:00:15
Processing layer-2             done | 00:00:15
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:31

Fitting 3 layers
Processing layer-1             done | 00:00:15
Processing layer-2             done | 00:00:14
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:30

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 3 layers
Processing layer-1             done | 00:00:15
Processing layer-2             done | 00:00:14
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:30

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete             

In [120]:
print(modelo_superaprendiz_dos_capas.data)

                           ft-m  ft-s  pt-m  pt-s
layer-1  bagging          14.63  0.16  0.94  0.11
layer-1  decision-tree     0.03  0.01  0.01  0.01
layer-1  extratrees        1.72  0.07  0.26  0.07
layer-1  random-forest     2.17  0.22  0.18  0.04
layer-2  bagging          14.38  0.14  0.81  0.08
layer-2  decision-tree     0.04  0.04  0.01  0.02
layer-2  extratrees        1.89  0.06  0.25  0.08
layer-2  random-forest     2.39  0.26  0.19  0.04



In [121]:
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_entrenamiento= modelo_superaprendiz_dos_capas.predict(X_trn)

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_entrenamiento = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_entrenamiento["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_entrenamiento["TIPO_DE_DATOS"]='Entrenamiento'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_entrenamiento


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  81.17
Exhaustividad:  82.42
Puntuación F1:  79.71
Exactitud:  82.42
Información Mutua Normalizada (NMI): 41.77
Índice Kappa de Cohen: 60.66


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,81.17,SuperAprendizdoscapas,Entrenamiento
1,Exhaustividad,82.42,SuperAprendizdoscapas,Entrenamiento
2,Puntuación F1,79.71,SuperAprendizdoscapas,Entrenamiento
3,Exactitud,82.42,SuperAprendizdoscapas,Entrenamiento
4,Información Mutua Normalizada (NMI),41.77,SuperAprendizdoscapas,Entrenamiento
5,Índice Kappa de Cohen,60.66,SuperAprendizdoscapas,Entrenamiento


In [122]:
# Predecir las etiquetas para los datos de prueba
resultados_df_superaprendiz_dos_capas = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

Y_pred_prueba = modelo_superaprendiz_dos_capas.predict(X_tst)

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)
print("Precisión: ", round(precision*100,2))
print("Exhaustividad: ", round(recall*100,2))
print("Puntuación F1: ", round(f1*100,2))
print("Exactitud: ", round(accuracy*100,2))
print("Información Mutua Normalizada (NMI):", round(nmi*100,2))
print("Índice Kappa de Cohen:", round(kappa*100,2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi=pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa=pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_superaprendiz_dos_capas_prueba = pd.concat([resultados_df_superaprendiz_dos_capas, df_precision, df_recall, df_f1, df_accuracy,df_nmi,df_kappa], ignore_index=True)
resultados_df_superaprendiz_dos_capas_prueba["MODELO"]='SuperAprendizdoscapas'
resultados_df_superaprendiz_dos_capas_prueba["TIPO_DE_DATOS"]='Prueba'
# Imprimir el DataFrame con los resultados
resultados_df_superaprendiz_dos_capas_prueba


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00
Precisión:  57.33
Exhaustividad:  64.01
Puntuación F1:  55.13
Exactitud:  64.01
Información Mutua Normalizada (NMI): 0.62
Índice Kappa de Cohen: 5.05


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.33,SuperAprendizdoscapas,Prueba
1,Exhaustividad,64.01,SuperAprendizdoscapas,Prueba
2,Puntuación F1,55.13,SuperAprendizdoscapas,Prueba
3,Exactitud,64.01,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),0.62,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,5.05,SuperAprendizdoscapas,Prueba


In [123]:
hiperparametros_superaprendiz_dos_capas = [] 
for nombre, estimador in estimadores:
    hiperparametros = estimador.get_params()
    hiperparametros_estimador = {'Estimador': nombre, 'Hiperparametros': hiperparametros}
    hiperparametros_superaprendiz_dos_capas.append(hiperparametros_estimador)


In [124]:
cadena_hiperparametros_superaprendiz_dos_capas = ', '.join([', '.join([f"{key}: {value}" for key, value in estimador.items()]) for estimador in hiperparametros_superaprendiz_dos_capas])

df_hiperparametros_superaprendiz_dos_capas = pd.DataFrame({
    'MÉTRICA': ['Mejores Hiperparametros'],
    'VALOR': [cadena_hiperparametros_superaprendiz_dos_capas],
    'MODELO': ['SuperAprendizdoscapas'],
    'TIPO_DE_DATOS': ['Hiperparametros del modelo']
})
resultados_df_superaprendiz_dos_capas = pd.concat([resultados_df_superaprendiz_dos_capas_prueba,resultados_df_superaprendiz_dos_capas_entrenamiento,df_hiperparametros_superaprendiz_dos_capas], ignore_index=True)
resultados_df_superaprendiz_dos_capas

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,57.33,SuperAprendizdoscapas,Prueba
1,Exhaustividad,64.01,SuperAprendizdoscapas,Prueba
2,Puntuación F1,55.13,SuperAprendizdoscapas,Prueba
3,Exactitud,64.01,SuperAprendizdoscapas,Prueba
4,Información Mutua Normalizada (NMI),0.62,SuperAprendizdoscapas,Prueba
5,Índice Kappa de Cohen,5.05,SuperAprendizdoscapas,Prueba
6,Precisión,81.17,SuperAprendizdoscapas,Entrenamiento
7,Exhaustividad,82.42,SuperAprendizdoscapas,Entrenamiento
8,Puntuación F1,79.71,SuperAprendizdoscapas,Entrenamiento
9,Exactitud,82.42,SuperAprendizdoscapas,Entrenamiento


### RED NEURONAL

In [125]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
np.random.seed(42)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_neural_network(X_train, y_train, input_size, hidden_size, num_classes, num_epochs=100, learning_rate=0.0015):
    # Convertir datos de entrenamiento a tensores de PyTorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    
    # Inicializar modelo y función de pérdida
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        # Forward pass y pérdida
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    return model,num_epochs,learning_rate

def evaluate_model(model, X_test, y_test):
    # Convertir datos de prueba a tensores de PyTorch
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Obtener las predicciones del modelo
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    # Calcular las métricas del modelo
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    precision = precision_score(y_test, predicted.numpy(), average='weighted')
    recall = recall_score(y_test, predicted.numpy(), average='weighted')
    f1 = f1_score(y_test, predicted.numpy(), average='weighted')

    print('Accuracy: {:.2f}%'.format(accuracy * 100))
    print("Precisión: ", precision)
    print("Exhaustividad (Recall): ", recall)
    print("Puntuación F1: ", f1)

X_train_array = X_trn.values
X_test_array = X_tst.values
Y_trn = Y_trn 
input_size = X_trn.shape[1]
num_classes = 4
hidden_size =155

# Supongamos que X_train, y_train, X_test, y_test son tus datos de entrenamiento y prueba
modelo_red_reuronal,num_epochs,learning_rate = train_neural_network(X_train_array , Y_trn, input_size, hidden_size, num_classes)
evaluate_model(modelo_red_reuronal, X_test_array , Y_tst)


Epoch [10/100], Loss: 1.0317
Epoch [20/100], Loss: 0.8806
Epoch [30/100], Loss: 0.8426
Epoch [40/100], Loss: 0.8295
Epoch [50/100], Loss: 0.8202
Epoch [60/100], Loss: 0.8132
Epoch [70/100], Loss: 0.8088
Epoch [80/100], Loss: 0.8057
Epoch [90/100], Loss: 0.8029
Epoch [100/100], Loss: 0.8005
Accuracy: 63.68%
Precisión:  0.5363515276298154
Exhaustividad (Recall):  0.6368159203980099
Puntuación F1:  0.5203210902432832


In [126]:
print("Tamaño de entrada:", input_size)
print("Tamaño de la capa oculta:", hidden_size)
print("Número de clases:", num_classes)
print("Número de épocas:", num_epochs)
print("Tasa de aprendizaje:", learning_rate)


Tamaño de entrada: 6
Tamaño de la capa oculta: 155
Número de clases: 4
Número de épocas: 100
Tasa de aprendizaje: 0.0015


In [127]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, normalized_mutual_info_score, cohen_kappa_score

# Predecir las etiquetas para los datos de entrenamiento
resultados_df_red_reuronal = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de entrenamiento a tensores de PyTorch
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_train_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_entrenamiento = predicted.numpy()

precision = precision_score(Y_trn, Y_pred_entrenamiento, average='weighted')
recall = recall_score(Y_trn, Y_pred_entrenamiento, average='weighted')
f1 = f1_score(Y_trn, Y_pred_entrenamiento, average='weighted')
accuracy = accuracy_score(Y_trn, Y_pred_entrenamiento)
nmi = normalized_mutual_info_score(Y_trn, Y_pred_entrenamiento)
kappa = cohen_kappa_score(Y_trn, Y_pred_entrenamiento)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_entrenamiento = pd.concat([resultados_df_red_reuronal, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_entrenamiento["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_entrenamiento["TIPO_DE_DATOS"] = 'Entrenamiento'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_entrenamiento


Precisión:  59.07
Exhaustividad:  62.28
Puntuación F1:  50.77
Exactitud:  62.28
Información Mutua Normalizada (NMI): 1.27
Índice Kappa de Cohen: 5.05


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,59.07,RedNeuronal,Entrenamiento
1,Exhaustividad,62.28,RedNeuronal,Entrenamiento
2,Puntuación F1,50.77,RedNeuronal,Entrenamiento
3,Exactitud,62.28,RedNeuronal,Entrenamiento
4,Información Mutua Normalizada (NMI),1.27,RedNeuronal,Entrenamiento
5,Índice Kappa de Cohen,5.05,RedNeuronal,Entrenamiento


In [128]:
# Predecir las etiquetas para los datos de prueba
resultados_df_red_reuronal_prueba = pd.DataFrame(columns=['MÉTRICA', 'VALOR'])

# Convertir datos de prueba a tensores de PyTorch
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)

# Obtener las predicciones del modelo
outputs = modelo_red_reuronal(X_test_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_prueba = predicted.numpy()

precision = precision_score(Y_tst, Y_pred_prueba, average='weighted')
recall = recall_score(Y_tst, Y_pred_prueba, average='weighted')
f1 = f1_score(Y_tst, Y_pred_prueba, average='weighted')
accuracy = accuracy_score(Y_tst, Y_pred_prueba)
nmi = normalized_mutual_info_score(Y_tst, Y_pred_prueba)
kappa = cohen_kappa_score(Y_tst, Y_pred_prueba)

print("Precisión: ", round(precision*100, 2))
print("Exhaustividad: ", round(recall*100, 2))
print("Puntuación F1: ", round(f1*100, 2))
print("Exactitud: ", round(accuracy*100, 2))
print("Información Mutua Normalizada (NMI):", round(nmi*100, 2))
print("Índice Kappa de Cohen:", round(kappa*100, 2))

# Crear un DataFrame para cada métrica
df_precision = pd.DataFrame({'MÉTRICA': ['Precisión'], 'VALOR': [round(precision*100, 2)]})
df_recall = pd.DataFrame({'MÉTRICA': ['Exhaustividad'], 'VALOR': [round(recall*100, 2)]})
df_f1 = pd.DataFrame({'MÉTRICA': ['Puntuación F1'], 'VALOR': [round(f1*100, 2)]})
df_accuracy = pd.DataFrame({'MÉTRICA': ['Exactitud'], 'VALOR': [round(accuracy*100, 2)]})
df_nmi = pd.DataFrame({'MÉTRICA': ['Información Mutua Normalizada (NMI)'], 'VALOR': [round(nmi*100, 2)]})
df_kappa = pd.DataFrame({'MÉTRICA': ['Índice Kappa de Cohen'], 'VALOR': [round(kappa*100, 2)]})

# Concatenar los DataFrames
resultados_df_red_reuronal_prueba = pd.concat([resultados_df_red_reuronal_prueba, df_precision, df_recall, df_f1, df_accuracy, df_nmi, df_kappa], ignore_index=True)
resultados_df_red_reuronal_prueba["MODELO"] = 'RedNeuronal'
resultados_df_red_reuronal_prueba["TIPO_DE_DATOS"] = 'Prueba'

# Imprimir el DataFrame con los resultados
resultados_df_red_reuronal_prueba


Precisión:  53.64
Exhaustividad:  63.68
Puntuación F1:  52.03
Exactitud:  63.68
Información Mutua Normalizada (NMI): 0.1
Índice Kappa de Cohen: 0.9


,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.64,RedNeuronal,Prueba
1,Exhaustividad,63.68,RedNeuronal,Prueba
2,Puntuación F1,52.03,RedNeuronal,Prueba
3,Exactitud,63.68,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),0.10,RedNeuronal,Prueba
5,Índice Kappa de Cohen,0.90,RedNeuronal,Prueba


In [129]:
# Crear un diccionario con los hiperparámetros y sus valores
hiperparametros = {
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_classes": num_classes,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate
}

# Convertir el diccionario en una cadena formateada para mostrar en el dataframe
cadena_hiperparametros = ', '.join([f"{key}={value}" for key, value in hiperparametros.items()])

# Crear un nuevo dataframe con los hiperparámetros
df_hiperparametros_red_neuronal = pd.DataFrame({
    'MÉTRICA': ['Hiperparámetros'],
    'VALOR': [cadena_hiperparametros],
    'MODELO': ['RedNeuronal'],
    'TIPO_DE_DATOS': ['Entrenamiento']
})

In [130]:
resultados_df_red_reuronal = pd.concat([resultados_df_red_reuronal_prueba,resultados_df_red_reuronal_entrenamiento,df_hiperparametros_red_neuronal], ignore_index=True)
resultados_df_red_reuronal

,MÉTRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precisión,53.64,RedNeuronal,Prueba
1,Exhaustividad,63.68,RedNeuronal,Prueba
2,Puntuación F1,52.03,RedNeuronal,Prueba
3,Exactitud,63.68,RedNeuronal,Prueba
4,Información Mutua Normalizada (NMI),0.1,RedNeuronal,Prueba
5,Índice Kappa de Cohen,0.9,RedNeuronal,Prueba
6,Precisión,59.07,RedNeuronal,Entrenamiento
7,Exhaustividad,62.28,RedNeuronal,Entrenamiento
8,Puntuación F1,50.77,RedNeuronal,Entrenamiento
9,Exactitud,62.28,RedNeuronal,Entrenamiento


### Métricas

In [131]:
Metricas_Modelos=pd.concat([resultados_df_knn,resultados_df_svc,resultados_df_tree,
                            resultados_df_gaussian,resultados_df_LDA,resultados_df_BG,
                            resultados_df_random,resultados_df_extra,resultados_df_ADA,
                            resultados_df_GD,resultados_df_XB,resultados_df_CB,
                            resultados_df_LIGHT,resultados_df_voting_hard,resultados_df_voting_soft,
                            resultados_df_stacking_lineal,resultados_df_stacking_nolineal,
                            resultados_df_weighted_voting,resultados_df_superaprendiz,
                            resultados_df_superaprendiz_dos_capas,resultados_df_red_reuronal],axis=0)
Metricas_Modelos = Metricas_Modelos.rename(columns={'MÉTRICA': 'METRICA'})
Metricas_Modelos ['METRICA'] = Metricas_Modelos ['METRICA'].apply(lambda x: unidecode(x))
#Metricas_Modelos= Metricas_Modelos[Metricas_Modelos["MODELO"].isin(modelos_seleccionados)]

In [132]:
filtrado=Metricas_Modelos.loc[Metricas_Modelos['MODELO'].str.contains('RandomForest')]
pd.set_option('display.max_colwidth', None)
filtrado

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,55.68,RandomForest,Prueba
1,Exhaustividad,63.52,RandomForest,Prueba
2,Puntuacion F1,54.45,RandomForest,Prueba
3,Exactitud,63.52,RandomForest,Prueba
4,Informacion Mutua Normalizada (NMI),0.4,RandomForest,Prueba
5,Indice Kappa de Cohen,3.74,RandomForest,Prueba
6,Precision,68.94,RandomForest,Entrenamiento
7,Exhaustividad,66.41,RandomForest,Entrenamiento
8,Puntuacion F1,57.98,RandomForest,Entrenamiento
9,Exactitud,66.41,RandomForest,Entrenamiento


In [133]:
Metricas_Modelos ['MODELO'].unique()

array(['KNeighbors', 'SVC', 'DecisionTree', 'NaiveBayes', 'LDA',
       'Bagging', 'RandomForest', 'ExtraTrees', 'AdaBoost',
       'GradientBoosting', 'XGB', ' XGB', 'CatBoost', 'LGBM',
       'VotingHard', 'VotingSoft', 'StackingLineal', 'Stackingnolineal',
       'VotingWeighted', 'SuperAprendiz', 'SuperAprendizdoscapas',
       'RedNeuronal'], dtype=object)

In [134]:
data_with_columns = Metricas_Modelos.to_dict(orient='records')

diccionario_dataframes = [
        {
            'dataTransformacion': data_with_columns,
            #'columnas': Metricas_Modelos.columns.tolist()
        }
    ]
with open("Metricas_Modelos.json", "w") as json_file:
    json.dump({"data": diccionario_dataframes}, json_file, indent=4)

    print("Los DataFrames han sido guardados en 'Metricas_Modelos.json'.")

Los DataFrames han sido guardados en 'Metricas_Modelos.json'.


In [135]:
Metricas_Modelos.to_csv('Metricas_Modelos1.csv',sep="|",index=False,encoding='utf-8')

In [136]:
Metricas_Modelos

,METRICA,VALOR,MODELO,TIPO_DE_DATOS
0,Precision,51.65,KNeighbors,Prueba
1,Exhaustividad,58.87,KNeighbors,Prueba
2,Puntuacion F1,53.6,KNeighbors,Prueba
3,Exactitud,58.87,KNeighbors,Prueba
4,Informacion Mutua Normalizada (NMI),0.09,KNeighbors,Prueba
...,...,...,...,...
8,Puntuacion F1,50.77,RedNeuronal,Entrenamiento
9,Exactitud,62.28,RedNeuronal,Entrenamiento
10,Informacion Mutua Normalizada (NMI),1.27,RedNeuronal,Entrenamiento
11,Indice Kappa de Cohen,5.05,RedNeuronal,Entrenamiento
